<img src="https://pa-legg.github.io/images/uwe_banner.png">

# UFCFFY-15-M Cyber Security Analytics 23-24

## Portfolio Assignment: Worksheet 1
## Investigate a web server application logs to identify malicious attack activity using Python data science libraries
---

For this task, the company **"UWEtech"** have enlisted your support as a security data analyst. They suspect some kind of attack on their web server application, however they are unable to diagnose what has happened exactly, or which of their users have caused the attack. The company have provided you with their recent log data records. You will need to complete a series of tasks to examine the data further, and then expand your investigation to uncover any malicious activity.

**Dataset**: You will be randomly issued a **unique dataset** based on your UWE username - **failure to use the dataset assigned to your username will result in a zero grade**. Please see the folder ***"Portfolio Assignment"*** under the Assignment tab on Blackboard for further detail related to the access and download of the necessary dataset.

### Assessment and Marking
---
The completion of this worksheet is worth **20%** of your portfolio assignment for the UFCFFY-15-M Cyber Security Analytics (CSA) module. Each question states the possible marks available for a correct answer. Where a question is worth more than 1 mark, a partial solution to the question may warrant partial marks.

### Submission Documents
---

Your submission for this task should include:

- **1 Jupyter Notebook file (*.ipynb)**

You should complete your work using the ipynb file provided (i.e., this document). Once you have completed your work, you should ensure that all code cells have been executed and then you should save your notebook. **Please note: Staff will NOT execute your notebook during marking. It is your responsibility to ensure that your saved notebook shows the code cell outputs as required.**

The deadline for your portfolio submission is **THURSDAY 2ND MAY @ 14:00**. This assignment is eligible for the [48-hour late submission window](https://www.uwe.ac.uk/study/academic-information/personal-circumstances/late-submission-window), however module staff will not be able to assist with any queries after the deadline.

Your portfolio submitted to Blackboard must contain 3 independent documents:

- ***STUDENT_ID-TASK1.ipynb*** (your iPYNB with all cells executed)
- ***STUDENT_ID-TASK2.ipynb*** (your iPYNB with all cells executed)
- ***STUDENT_ID-TASK3.pdf*** (a PDF report of your research investigation)

### Contact
---

Questions about this assignment should be directed to your module leader (Phil.Legg@uwe.ac.uk). You should use the [online Q&A form](https://forms.office.com/e/yxFJZDraRG) to ask questions related to this module and this assignment, as well as utilising the on-site teaching sessions.

---

# Student ID: 23008852

- **By submitting this assignment to Blackboard as part of your portfolio, I declare that the submission is my own work.**

***

In [1]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pandas as pd
import numpy as np
import networkx as nx
import seaborn as sns
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 20)
import warnings
warnings.filterwarnings('ignore')

In the cell below, you will need to change `data_file` to your own specific data filename. The example data file is purely to demonstrate some initial steps for your investigation and should not be used.

In [3]:
data_file = './y232-wang_2024jan_dataset.txt'

In [ ]:
# Load in the data set as required
data = pd.read_csv(data_file, sep=r'\s+')
# original
# data = pd.read_csv(data_file, delim_whitespace=True)
temp_df = data[data.columns[:-1]]
temp_df.columns = data.columns[1:]
data = temp_df
data['datetime'] = pd.to_datetime(data['date'] + " " + data['time'])
data

### Question 1: Show how often each of the unique machines (defined by client IP address 'c-ip') have accessed this web server application? (1 Mark)

*Hint: Think about how you would count the occurrence of all possible values within a Dataframe column.*

In [ ]:
# Q1 ANSWER
# Count the unique client IP addresses and print the outputs in order
def factor_col(col):
    factor = pd.Categorical(col)
    return pd.Series(factor).value_counts(sort=False)

df = pd.DataFrame(data['c-ip'])

# count the numbers of each client IP address by following the original order in the array

f_cip = pd.DataFrame(factor_col(df['c-ip']))

# c-ips are sorted in a desending order
sort_f_cip = f_cip.sort_values('count', ascending=False)
sort_f_cip

### Question 2: Show how often each of the possible unique URLs (defined by 'cs(Referer)') have been accessed within the dataset? (1 Mark)

*Hint: Think about how you would count the occurrence of all possible values within a Dataframe column.*

In [ ]:

# Q2 ANSWER
# Count the possible unique URLs 'cs(Referer)'
def factor_col(col):
    factor = pd.Categorical(col)
    return pd.Series(factor).value_counts(sort=False)

# Count the numbers of the possible unique URLs 'cs(Referer)'
# Including '-'
df = pd.DataFrame(data['cs(Referer)'])
cre_ct = factor_col(df['cs(Referer)'])

# Remove '-' URLs

rrt_df = data[data['cs(Referer)'] != "-"]

# Count the numbers of the possible unique URLs 'cs(Referer)'
# Without '-'
cre = rrt_df['cs(Referer)']
cre_ct_02 = pd.Series(cre).value_counts()
pd.DataFrame(cre_ct_02)

### Question 3: Show how often each client IP has accessed each URL? (1 Mark)

*Hint: Think about how you would count the occurrence of all possible pairs of values within multiple Dataframe columns.*

In [ ]:
# Q3 ANSWER
# all '-' in URLs are removed in rrt_df
xtab = pd.crosstab(rrt_df['c-ip'], rrt_df['cs(Referer)'])
xtab

### Question 4: Filter the data to show all rows where 'cs-uri-query' starts with the string 'v='. (1 Mark)

In [ ]:
# Q4 ANSWER

# Extract all rows where 'cs-uri-query' starts with the string 'v='
df = pd.DataFrame(data)

df['cuq'] = df['cs-uri-query']
df['cuq'] = df['cuq'].str.contains('v=', regex=True)
df[df['cuq'] == True ]


### Question 5: Filter the data to show all rows where 'cs(User-Agent)' contain the string 'Win64? (1 Mark)

In [ ]:
# Q5 ANSWER
df = pd.DataFrame(data)
# Extract all rows where 'cs(User-Agent)' contain the string 'Win64'
df['cua'] = df['cs(User-Agent)']
df['cua_win64'] = df['cua'].str.contains('Win64', regex=True )
df[df['cua_win64'] == True ]

### Question 6: Filter the data to show all rows where 'sc-status' is equal to 404, before 07:00AM? (1 Mark)

In [ ]:
# Q6 ANSWER
# Extract all rows where 'sc-status' is equal to 404, before 07:00AM
df = pd.DataFrame(data)
ss_df = df[(df['sc-status'] == 404) & (pd.to_datetime(df['time']) < pd.to_datetime('07:00:00')) ]
ss_df

### Question 7: Plot a Bar Chart showing the number of entries for each value in 'sc-status'. (1 Mark)

*Hint: Matplotlib and Pandas are closely linked - some pandas functions have matplotlib functions embedded. Read the documentation for more detail.*


In [ ]:
# Q7 ANSWER

plt.figure(figsize=(8,7))
sns.countplot(x='sc-status',data=data, hue='sc-status', palette='rocket_r')
plt.title('Count of sc-status')  # Add title
plt.xlabel('Count')
plt.ylabel('SC-STATUS')
plt.show()

### Question 8: Plot a line chart that shows the number of records (Y-axis) vs each day within the dataset (X-axis). (1 Mark)

In [ ]:
df = pd.DataFrame(data)
# sort by date
da_ct = factor_col(df['date'])
da=da_ct.to_dict()
da

In [ ]:
# Q8 ANSWER

# Convert the dictionary 'da' into a DataFrame
dff = pd.DataFrame(list(da.items()), columns=['Date', 'Value'])

# Set the 'Date' column as datetime type
dff['Date'] = pd.to_datetime(dff['Date'])

# Create line plot using seaborn
plt.figure(figsize=(14, 6))
sns.lineplot(x='Date', y='Value', data=dff, marker='*', markersize=17)
plt.title('Each day within the dataset ')
plt.xlabel('Date')
plt.ylabel('Number of records')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

### (Advanced) Question 9: Filter and Plot a Network graph using the NetworkX library based on the criteria below (4 Marks)

* Filter the data to only the 20 most active IP addresses and plot this using a Network chart **(1 Mark)**.
* (Advanced) Scale each client node within the chart based on the amount of activity **(3 Marks)**.

In [ ]:
# Q9 ANSWER
# Count the numbers
def factor_col(col):
    factor = pd.Categorical(col)
    return pd.Series(factor).value_counts(sort=False)

# ----------------------------------------------------------------------------------------------------------
# Q9.1 Filter the data to only the 20 most active IP addresses and plot this using a Network chart (1 Mark).
# ----------------------------------------------------------------------------------------------------------

df = pd.DataFrame(data['c-ip'])

# count the numbers of each client IP address by following the original order in the array
f_cip = pd.DataFrame(factor_col(df['c-ip']))

s_cip = data['s-ip'].unique()
s_cip


In [ ]:
s_cip = {cip: 1 for cip in s_cip}
s_cip

In [ ]:
# c-ips are sorted in a desending order
sort_f_cip = f_cip.sort_values('count', ascending=False)


# Top 20 Active IPs, sorted in descending order
net_ip = sort_f_cip[:20]
graph_dict = net_ip.to_dict()
node_data = graph_dict['count']
node_data

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt


# Create a NetworkX graph
G = nx.Graph()

# Add nodes to the graph and set node sizes based on counts
for node, count in node_data.items():
    G.add_node(node, count=count)

# Add s_cip node(s)
for cip in s_cip:
    G.add_node(cip)

# Connect each node in node_data to s_cip node(s)
for node in node_data.keys():
    for cip in s_cip:
        G.add_edge(node, cip)

# Draw the network chart
nx.draw(G, with_labels=True, labels={node: f"{node}\nCount: {node_data[node]}" if node in node_data else f"{node}\nCount: 1" for node in G.nodes()},
        node_color='skyblue', node_size=500, font_size=6, edge_color='gray')

plt.title("Network Chart with Connections to s_ip")
plt.show()

In [ ]:

# Create a NetworkX graph
G = nx.Graph()

# Add nodes to the graph and set node sizes based on counts
for node, count in node_data.items():
    # Adjust the size based on count
    G.add_node(node, count=count, size=count * 10)

# Add s_cip node(s)
for cip, size in s_cip.items():
    G.add_node(cip, size=size)

# Connect each node in node_data to s_cip node(s)
for node in node_data.keys():
    for cip in s_cip.keys():
        G.add_edge(node, cip)

# Draw the network chart
nx.draw(G, with_labels=True, labels={node: f"{node}\nCount: {node_data[node]}" if node in node_data else f"{node}\nCount: 1" for node in G.nodes()},
        node_color='coral', node_size=[G.nodes[node].get('size', 100) for node in G.nodes()], font_size=6, edge_color='gray')

plt.title("Scale Each Client Node Based On The Amount Of Activity")
plt.show()


### Question 10:  Investigate the dataset further to uncover the suspicious activity (8 marks)

This unguided question will be graded against the following criteria:

- Identifying all suspicious activity within your dataset **(3 Marks)**
- Analytical reasoning of investigating why this activity is suspicious **(3 Marks)**
- Clarity and presentation **(2 Marks)**

You should state all suspicious IP addresses that you have identified as part of your conclusion, and you should explain in clear written English how you have uncovered this information, based on how you have used Python code for data investigation. This should be clear and concise, and you only need to include code that helped you to solve the challenge.

# 1 Identifying all suspicious activity within your dataset


## 1.1 Step 1. Check orginal data
- Check Original data to find some suspicious elements as a starting point for investigation.
- Check number of daily data/200/404

In [ ]:
data

### Step 1.2  Calculate numbers of 'GET' & 'POST'

In [ ]:
# Count the numbers
def factor_col(col):
    factor = pd.Categorical(col)
    return pd.Series(factor).value_counts(sort=False)
    
df = pd.DataFrame(data)

# count the numbers of each client IP address by following the original order in the array
f_cs = pd.DataFrame(factor_col(df['cs-method']))
f_cs

### Step 1.3 Check Successful Responses 200
- So the successful 'GET' & 'POST' methods can viewed 

In [ ]:
# Extract all rows where 'sc-status' is 200
csdf = df[(df['sc-status'] == 200)]
csdf

### Step 1.4 Check the number of 'GET' & 'POST'

In [ ]:
# Count the numbers
def factor_col(col):
    factor = pd.Categorical(col)
    return pd.Series(factor).value_counts(sort=False)
    

# count the numbers of each client IP address by following the original order in the array
f_cs02 = pd.DataFrame(factor_col(csdf['cs-method']))
f_cs02

### Step 1.5 List the suspicious IP addresses
- `194.245.32.53` has the highest counts on 'GET' mthod.
- `92.215.11.248` and `94.177.126.101` have the near numbers respectively 131 and 128.

In [ ]:
# Count the numbers in a descending order
def factor_coll(col):
    factor = pd.Categorical(col)
    return pd.Series(factor).value_counts(sort=True)

csip = pd.DataFrame(factor_coll(csdf['c-ip']))
csip

# 2 Analytical reasoning of investigating why this activity is suspicious

## 2.1 Investigate activities of the suspicious ip `194.245.32.53`

- Check the activities of `194.245.32.53`

In [ ]:
# Extract all rows where 'sc-status' is 200
ip01 = df[(df['c-ip'] == '194.245.32.53')]
ip01

## 2.2 Check the most interactive date
- `2023-01-29` was the date `194.245.32.53` was most active.

In [ ]:
# Count the numbers in a descending order
def factor_coll(col):
    factor = pd.Categorical(col)
    return pd.Series(factor).value_counts(sort=True)

ip01_dates = pd.DataFrame(factor_coll(ip01['date']))
ip01_dates

### 2.3 Investigate `2023-01-29`
- I will check `time-taken` to know the time of this IP `194.245.32.53` spent on 'GET' or 'POST'

In [ ]:
ip02 = ip01[ip01['date'] == '2023-01-29']
ip02

### 2.3 Check the frequency of `cs-method`
- IP `194.245.32.53` had retrieved data from the server most of the time.

In [ ]:
# Count the numbers in a descending order
def factor_coll(col):
    factor = pd.Categorical(col)
    return pd.Series(factor).value_counts(sort=True)

ip03 = pd.DataFrame(factor_coll(ip02['cs-method']))
ip03

### 2.4 Check POST Method
- According to the result below, the URL of the requested URL `https://uwetech.co.uk/login.aspx` has been changed permanently. This indicates that the website is employing some form of redirection or user session management
- Same IP address `194.245.32.53` login as two different users `rl236451` and `mr257125` on `2023-01-29`.

In [ ]:
ip04 = ip02[ip02['cs-method'] == 'POST']
ip04

# 3 Conclusion
- In accordance with the investigation, IP address `194.245.32.53` had unusual highest number of activities(188 'GET' activities in January).
- `2023-01-29` is the date that `194.245.32.53` was most active.
- `194.245.32.53` had retrieved data from the server most of the time (81 'GET' activities on `2023-01-29`).
- The URL of the requested URL `https://uwetech.co.uk/login.aspx` has been changed permanently. This indicates that the website is employing some form of redirection or user session management. It could be worth further investigation for this suspicious login event.
- Same IP address `194.245.32.53` login as two different users `rl236451` and `mr257125` on `2023-01-29`.

# Appendix

### Figure 1 - Line Chart of Daily Data 

In [ ]:

# Convert the dictionary 'da' into a DataFrame
dff = pd.DataFrame(list(da.items()), columns=['Date', 'Value'])

# Set the 'Date' column as datetime type
dff['Date'] = pd.to_datetime(dff['Date'])

# Create line plot using seaborn
plt.figure(figsize=(14, 6))
sns.set_theme(style="darkgrid")

# sns.lineplot(x='Date', y='Value', data=jj, marker='o')
sns.lineplot(x='Date', y='Value', data=dff, marker='v', markersize=13)


plt.title('Daily data')
plt.xlabel('Date')
plt.ylabel('Number of records')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

### Figure 2 - Dictionary of suspicious activities of `194.245.32.53`

In [ ]:
import datetime

# Initialize an empty dictionary
date_dict = {}

# Start date
start_date = datetime.date(2023, 1, 1)

# Iterate over the range from 1 to 31
for i in range(1, 31):
    # Calculate the date for the current iteration
    current_date = start_date + datetime.timedelta(days=i-1)
    if i < 10:
        xx = df[(df['c-ip'] == '194.245.32.53') &(df['date'] == '2023-01-0'+str(i))]
    else:
        xx = df[(df['c-ip'] == '194.245.32.53')&(df['date'] == '2023-01-'+str(i))]
    len_x = len(xx)
    # Add the date and corresponding number to the dictionary
    date_dict[current_date.strftime('%Y-%m-%d')] = len_x

# Print the dictionary
for key, value in date_dict.items():
    print(key, value)


### Figure 3 - Line Chart of Suspicious Sctivities of `194.245.32.53`

In [ ]:
# Convert the dictionary 'date_dict' into a DataFrame
jj = pd.DataFrame(list(date_dict.items()), columns=['Date', 'Value'])

# Set the 'Date' column as datetime type
jj['Date'] = pd.to_datetime(jj['Date'])

# Create line plot using seaborn
plt.figure(figsize=(14, 6))
sns.set_theme(style="darkgrid", palette="Spectral")
sns.lineplot(x='Date', y='Value', data=jj, marker='o', markersize=10)
plt.title('Activit of `194.245.32.53`')
plt.xlabel('Date')
plt.ylabel('Number of records')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

# Reference
- [Python Seaborn Tutorial For Beginners: Start Visualizing Data](https://www.datacamp.com/tutorial/seaborn-python-tutorial)
- [NetworkX - Creating a graph](https://networkx.org/documentation/stable/tutorial.html#nx-guides)
- [HTTP response status codes](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status#successful_responses)
- [Cross-Origin Resource Sharing (CORS)](https://developer.mozilla.org/en-US/docs/Web/HTTP/CORS)